In [1]:
#  Demonstrating Dropout behavior in all 3 Keras APIs
# ====================================================

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#  DATASET
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

# Normalize pixel values (0-255 -> 0-1)
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

# Flatten images (28x28 -> 784)
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)


# SEQUENTIAL API
# ------------------------------------------------------
seq_model = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=(784,)),   # Hidden layer 1
    layers.Dropout(0.5),                                        # Dropout
    layers.Dense(128, activation='relu'),                       # Hidden layer 2
    layers.Dense(10, activation='softmax')                      # Output layer (10 classes)
])

# Compile model
seq_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Train model
print("\n Training Sequential Model...")
seq_model.fit(X_train, y_train, epochs=2, batch_size=128, verbose=1)

# Evaluate (dropout is automatically off here unlike sub class apis)
print("\n Sequential Model Evaluation:")
seq_model.evaluate(X_test, y_test)


# FUNCTIONAL API
# ------------------------------------------------------
# Define inputs
inputs = keras.Input(shape=(784,))

# Pass through layers manually
x = layers.Dense(256, activation='relu')(inputs) #put inputs as func in 1st
x = layers.Dropout(0.5)(x)             # Dropout auto handled
x = layers.Dense(128, activation='relu')(x) 
outputs = layers.Dense(10, activation='softmax')(x)

# Build model
func_model = keras.Model(inputs=inputs, outputs=outputs, name="functional_model") #define whats input and output in case of multiple stuffs

# Compile model
func_model.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

# Train model
print("\n Training Functional Model...")
func_model.fit(X_train, y_train, epochs=2, batch_size=128, verbose=1)

# Evaluate (dropout is automatically off here too since unlike subclass api here kerars does it automatically)
print("\n Functional Model Evaluation:")
func_model.evaluate(X_test, y_test)


#  SUBCLASSED MODEL API
# ------------------------------------------------------
class MyCustomModel(keras.Model):
    def __init__(self): #initialize layers
        super().__init__()  # inherit from keras.Model for alll kerars methods like compile,predeicts and etc etc.......
        # Define layers
        self.dense1 = layers.Dense(256, activation='relu')
        self.dropout = layers.Dropout(0.5)
        self.dense2 = layers.Dense(128, activation='relu')
        self.out = layers.Dense(10, activation='softmax')

    # Custom forward pass
    def call(self, inputs, training=False):
        x = self.dense1(inputs)
        if training:                     # Dropout only during training
            x = self.dropout(x)
        x = self.dense2(x)
        return self.out(x)

# Instantiate and compile model
sub_model = MyCustomModel()
sub_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Train model (Dropout applied because training=True automatically inside fit)
print("\n Training Subclassed Model...")
sub_model.fit(X_train, y_train, epochs=2, batch_size=128, verbose=1)

# Evaluate (Dropout automatically not applied because training=False)
print("\n Subclassed Model Evaluation:")
sub_model.evaluate(X_test, y_test)


#  Final Check: Prediction Comparison
# ------------------------------------------------------
print("\n Checking Dropout Behavior During Prediction")
print("Sequential:", seq_model(X_test[:1], training=False))
print("Functional:", func_model(X_test[:1], training=False))
print("Subclassed:", sub_model(X_test[:1], training=False))


C:\Users\chala\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



 Training Sequential Model...
Epoch 1/2
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7796 - loss: 0.6197
Epoch 2/2
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8352 - loss: 0.4534

 Sequential Model Evaluation:
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8452 - loss: 0.4199

 Training Functional Model...
Epoch 1/2
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7768 - loss: 0.6227
Epoch 2/2
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8386 - loss: 0.4485

 Functional Model Evaluation:
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8556 - loss: 0.4028

 Training Subclassed Model...
Epoch 1/2
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7768 - loss: 0.6234
Epoch 2/2
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8359 - loss: 0.4546

 Subclassed Model Evaluation:
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8521 - loss: 0.4024

🔍 Checking Dropout Behavior During Prediction
Sequential: tf.Tensor(
[[7.18816